# Bat Algorithm

In [1]:
import torch
from torch import normal, randint, rand, multinomial
from math import pi, cos, sin, gamma
from tqdm import tqdm

In [2]:
from obj_f import ackley_f, weierstrass_f

In [3]:
from random import choices

In [4]:
def argmin(lst):
    return min(range(len(lst)), key=lst.__getitem__)

In [8]:
class bat_algorithm(object):
    def __init__(self, function, pop_size=50, low_bound=-20, up_bound=20, dimension=10,
                 fmin=0, fmax=1, A0=1, alpha=0.9, r0=1, r=0.1, sigma=0.5, **kwargs):

        self.pop_size = pop_size
        self.low_bound = low_bound
        self.up_bound = up_bound
        self.dimension = dimension
        self.obj_f = function

        self.fmin = fmin
        self.fmax = fmax

        self.v0 = [torch.zeros((dimension, 1)) for _ in range(pop_size)]
        self.A0 = A0
        self.alpha = alpha
        self.r0 = r0
        self.r = r
        self.sigma = sigma
        
    def forward(self, iteration_time, **kwargs):
        v = self.v0[:]
        A_t = self.A0

        init_points_list = self.random_generate_sampe(size=self.pop_size)
        points_list = init_points_list[:]
        
        function_values = [self.obj_f(sub_point, **kwargs) for sub_point in init_points_list]
        
        
        global_best_value_each_step = []
        global_best_point_each_step = []
        
        global_best_index = argmin(function_values)
        global_best_value_each_step.append(function_values[global_best_index])
        global_best_point_each_step.append(points_list[global_best_index])
        
        for t in tqdm(range(iteration_time)):
            if t > 0:
                A_t *= self.alpha
                r_t = self.r0 * (1 - torch.exp(torch.Tensor([- self.r * t])))
            else:
                A_t = self.A0
                r_t = self.r0 * (1 - torch.exp(torch.Tensor([- self.r * 1])))

            for bat_index in range(self.pop_size):
                new_point, v[bat_index] = self.generate_new_point(points_list[bat_index], v[bat_index], global_best_point_each_step[-1])
                if rand(1) > r_t:
                    new_point = global_best_point_each_step[-1] + normal(0, self.sigma, (self.dimension, 1))
                new_point = self.adj_to_domain(new_point)
                
                function_value = self.obj_f(new_point, **kwargs)
                
                if function_value < global_best_value_each_step[-1] and rand(1) < A_t:
                    function_values[bat_index] = function_value
                    points_list[bat_index] = new_point

            global_best_index = argmin(function_values)
            
            global_best_value_each_step.append(function_values[global_best_index])
            global_best_point_each_step.append(points_list[global_best_index])
        
        return torch.Tensor(global_best_value_each_step).tolist(), global_best_point_each_step
 
    def generate_new_point(self, point, velocity, current_best_point):
        f = self.fmin + (self.fmax - self.fmin) * rand(self.dimension, 1)
        updated_velocity = velocity + f * (point - current_best_point)
        updated_point = point + updated_velocity
        return updated_point, updated_velocity

        
    def adj_to_domain(self, x):
        low_index = x[:,0] < self.low_bound
        x[low_index,0] = self.low_bound
        up_index = x[:,0] > self.up_bound
        x[up_index,0] = self.up_bound
        return x
    
    def random_generate_sampe(self, size, *arg):
        init_points = (rand(self.dimension, size) - 1) * 20
        init_points = list(torch.split(init_points, split_size_or_sections=1, dim=1))
        return init_points

In [16]:
bat = bat_algorithm(pop_size=50,function=weierstrass_f,fmax=2,alpha=0.99,sigma=0.01,r=0.1,r0=1)

In [18]:
a, b = bat.forward(50)

100%|██████████| 50/50 [00:08<00:00,  5.72it/s]


In [4]:
# (版本一)
# class bat_algorithm(object):
#     def __init__(self, function, pop_size=50, low_bound=-20, up_bound=20, dimension=10,
#                  fmin=0, fmax=1, A0=1, alpha=0.9, r0=1, r=0.1, sigma=0.5, **kwargs):

#         self.pop_size = pop_size
#         self.low_bound = low_bound
#         self.up_bound = up_bound
#         self.dimension = dimension
#         self.obj_f = function

        
#         self.fmin = fmin
#         self.fmax = fmax

#         self.v0 = [torch.zeros((dimension, 1)) for _ in range(pop_size)]
#         self.v = self.v0[:]
        
#         self.A0 = A0
#         self.A_t = A0
#         self.alpha = alpha
        
#         self.r0 = r0
#         self.r_t = r0
#         self.r = r
#         self.sigma = sigma
        
#         self.iteration = 0
        
#         init_points_list = self.random_generate_sampe(size=self.pop_size)
#         self.points_list = init_points_list[:]
        
#         init_function_values_list = [self.obj_f(sub_point, **kwargs) for sub_point in init_points_list]
#         self.function_values = torch.Tensor(init_function_values_list)
        
#         self.global_best_value_each_step = []
#         self.global_best_point_each_step = []
        
#         self.global_best_index = torch.argmin(self.function_values)
#         self.global_best_value_each_step.append(self.function_values[self.global_best_index])
#         self.global_best_point_each_step.append(self.points_list[self.global_best_index])
        
        
#     def forward(self, iteration_time, **kwargs):

#         for t in tqdm(range(iteration_time)):
#             self.iteration += 1
#             if self.iteration > 0:
#                 self.A_t *= self.alpha
#                 self.r_t = self.r0 * (1 - torch.exp(torch.Tensor([- self.r * self.iteration])))

#             for bat_index in range(self.pop_size):
#                 new_point, self.v[bat_index] = self.generate_new_point(self.points_list[bat_index], self.v[bat_index], self.global_best_point_each_step[-1])
#                 if rand(1) > self.r_t:
#                     # new_point = self.global_best_point_each_step[-1] + self.sigma * rand(self.dimension, 1)
#                     new_point = self.global_best_point_each_step[-1] + normal(0, self.sigma, (self.dimension, 1))
#                 new_point = self.adj_to_domain(new_point)
                
#                 function_value = self.obj_f(new_point, **kwargs)
                
#                 if function_value < self.function_values[bat_index] and rand(1) < self.A_t:
#                     self.function_values[bat_index] = function_value
#                     self.points_list[bat_index] = new_point

#             self.global_best_index = torch.argmin(self.function_values)
            
#             self.global_best_value_each_step.append(self.function_values[self.global_best_index])
#             self.global_best_point_each_step.append(self.points_list[self.global_best_index])
            
 
#     def generate_new_point(self, point, velocity, current_best_point):
#         f = self.fmin + (self.fmax - self.fmin) * rand(self.dimension, 1)
#         updated_velocity = velocity + f * (point - current_best_point)
#         updated_point = point + updated_velocity
#         return updated_point, updated_velocity

        
#     def adj_to_domain(self, x):
#         low_index = x[:,0] < self.low_bound
#         x[low_index,0] = self.low_bound
#         up_index = x[:,0] > self.up_bound
#         x[up_index,0] = self.up_bound
#         return x
    
#     def random_generate_sampe(self, size, *arg):
#         init_points = (rand(self.dimension, size) - 1) * 20
#         init_points = list(torch.split(init_points, split_size_or_sections=1, dim=1))
#         return init_points

In [143]:
# (版本二)
# class bat_algorithm(object):
#     def __init__(self, function, pop_size=50, low_bound=-20, up_bound=20, dimension=10,
#                  fmin=0, fmax=1, A0=1, alpha=0.9, r0=1, r=0.1, sigma=0.5, **kwargs):

#         self.pop_size = pop_size
#         self.low_bound = low_bound
#         self.up_bound = up_bound
#         self.dimension = dimension
#         self.obj_f = function

#         self.fmin = fmin
#         self.fmax = fmax

#         self.v0 = [torch.zeros((dimension, 1)) for _ in range(pop_size)]
#         self.A0 = A0
#         self.alpha = alpha
#         self.r0 = r0
#         self.r = r
#         self.sigma = sigma
        
#     def forward(self, iteration_time, **kwargs):
        
#         v = self.v0[:]
#         self.A_t = A0

#         init_points_list = self.random_generate_sampe(size=self.pop_size)
#         points_list = init_points_list[:]
        
#         function_values = [self.obj_f(sub_point, **kwargs) for sub_point in init_points_list]
        
        
#         self.global_best_value_each_step = []
#         self.global_best_point_each_step = []
        
#         global_best_index = argmin(function_values)
#         global_best_value_each_step.append(function_values[global_best_index])
#         global_best_point_each_step.append(points_list[global_best_index])
        
#         for t in tqdm(range(iteration_time)):
#             if t > 0:
#                 A_t *= self.alpha
#                 r_t = self.r0 * (1 - torch.exp(torch.Tensor([- self.r * t])))
#             else:
#                 A_t = self.A0
#                 r_t = self.r0 * (1 - torch.exp(torch.Tensor([- self.r * 1])))
#             # for bat_index in range(self.pop_size):
                
#             new_points_list, self.v = self.generate_new_point(self.points_list, self.v, self.global_best_point_each_step[-1])
#             # new_points_list = [torch.clamp(sub_tensor, self.low_bound, self.up_bound) for sub_tensor in new_points_list]
#             # new_points_list = list(map(torch.clamp, new_points_list, [self.low_bound]*self.pop_size, [self.up_bound]*self.pop_size))
            
#             rand_index = torch.where(rand((self.pop_size)) > self.r_t)[0]
#             for k in rand_index:
#                 if k != self.global_best_index:
#                     new_points_list[k] = self.random_move(self.global_best_point_each_step[-1])
            
#             # new_points_list = list(map(torch.clamp, new_points_list, [self.low_bound]*self.pop_size, [self.up_bound]*self.pop_size))
#             new_points_list = list(map(self.adj_to_domain, new_points_list))
#             temp_function_values = torch.Tensor([self.obj_f(sub_point, **kwargs) for sub_point in new_points_list])
            
#             change_index = temp_function_values <= self.function_values
            
            
#             for sub_index in change_index:
#                 if rand(1) < self.A_t:
#                     self.function_values[sub_index] = temp_function_values[sub_index]
#                     self.points_list[sub_index] = new_points_list[sub_index]
            
#             min_index = torch.argmin(self.function_values)
#             self.global_best_value_each_step.append(self.function_values[min_index])
#             self.global_best_point_each_step.append(self.points_list[min_index])
        
# #     def generate_new_point(self, point_list, velocity_list, current_best_point):
# #         f = self.fmin + (self.fmax - self.fmin) * rand(self.dimension)
# #         for i in range(self.dimension):
# #             velocities_list[i] += (f[i]*(points_list[i] - current_best_point))
# #             points_list[i] += velocities_list[i]
# #         return points_list, velocities_list
    
#     def generate_new_point(self, points_list, velocities_list, current_best_point):
#         for i in range(self.pop_size):
#             f = self.fmin + (self.fmax - self.fmin) * rand(self.dimension,1)
            
#             velocities_list[i] += (f*(points_list[i] - current_best_point))
#             points_list[i] += velocities_list[i]
#         return points_list, velocities_list
    
#     def random_move(self, current_best_point):
#         result = current_best_point + normal(0, self.sigma, size=(self.dimension, 1))# * self.A_t
#         return result 

        
#     def adj_to_domain(self, x):
#         low_index = x[:,0] < self.low_bound
#         x[low_index,0] = self.low_bound
#         up_index = x[:,0] > self.up_bound
#         x[up_index,0] = self.up_bound
#         return x
    
#     def random_generate_sampe(self, size, *arg):
#         init_points = (rand(self.dimension, size) - 1) * 20
#         init_points = list(torch.split(init_points, split_size_or_sections=1, dim=1))
#         return init_points